# Cluster Interpretation: Mental Health in Tech 2016

## 1. Load Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

RUN = 'run_02'

df = pd.read_csv(f'../../data/{RUN}/clustered/clustered_data.csv')

## 2. Cluster Overview

In [ ]:
cluster_counts = df['cluster'].value_counts().sort_index()

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
colors = plt.cm.tab10(range(len(cluster_counts)))

axes[0].bar(cluster_counts.index, cluster_counts.values, color=colors)
axes[0].set_xlabel('Cluster')
axes[0].set_ylabel('Number of Participants')
axes[0].set_title('Cluster Sizes')
axes[0].set_xticks(cluster_counts.index)
for i, v in enumerate(cluster_counts.values):
    axes[0].text(i, v + 10, str(v), ha='center', fontweight='bold')

axes[1].pie(cluster_counts.values, labels=[f'Cluster {i}' for i in cluster_counts.index],
            autopct='%1.1f%%', colors=colors, startangle=90)
axes[1].set_title('Cluster Distribution')

plt.tight_layout()
os.makedirs(f'../../plots/{RUN}', exist_ok=True)
plt.savefig(f'../../plots/{RUN}/cluster_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

## 3. Define Feature Categories

In [8]:
feature_cols = [col for col in df.columns if col != 'cluster']

used_features = set()

def categorize(cols, keywords, exclude_keywords=None):
    result = []
    for col in cols:
        if col in used_features:
            continue
        if any(x in col for x in keywords):
            if exclude_keywords and any(x in col.lower() for x in exclude_keywords):
                continue
            result.append(col)
            used_features.add(col)
    return result

# 1. Demographic
demographic_features = categorize(feature_cols, 
    ['age', 'gender', 'country_live', 'country_work'])

# 2. Workplace
workplace_features = categorize(feature_cols,
    ['self-employed', 'employees', 'remotely', 'position_', 'tech company'])

# 3. Mental Health Status (personal)
mental_health_status = categorize(feature_cols,
    ['diagnosed_', 'currently have', 'in the past', 'family history', 
     'sought treatment', 'interferes with your work', 'medical professional'])

# 4. Previous Employer
previous_employer = categorize(feature_cols, ['previous'])

# 5. Current Employer Support
employer_support = categorize(feature_cols,
    ['provide mental health benefits', 'know the options', 'formally discussed', 
     'offer resources', 'anonymity protected', 'takes mental health as seriously',
     'employer'])

# 6. Attitudes & Stigma (everything else workplace-related)
attitudes_features = categorize(feature_cols,
    ['comfortable', 'willing', 'negative', 'hurt your career', 'bring up',
     'observed', 'coworker', 'supervisor', 'team member', 'identified'])

uncategorized = [col for col in feature_cols if col not in used_features]

# Summary
categories = {
    'Demographic': demographic_features,
    'Workplace': workplace_features,
    'Mental Health Status': mental_health_status,
    'Current Employer Support': employer_support,
    'Attitudes & Stigma': attitudes_features,
    'Previous Employer': previous_employer,
    'Other': uncategorized
}

for cat_name, cat_features in categories.items():
    if cat_features:
        print(f"{cat_name}: {len(cat_features)} features")

Demographic: 24 features
Workplace: 10 features
Mental Health Status: 22 features
Current Employer Support: 26 features
Attitudes & Stigma: 24 features
Previous Employer: 49 features
Other: 4 features


## 4. Heatmap: Cluster Means for All Features

In [ ]:
cluster_means = df.groupby('cluster')[feature_cols].mean()
cluster_means_norm = (cluster_means - cluster_means.mean()) / cluster_means.std()

top_features = cluster_means.var().nlargest(30).index.tolist()

fig, ax = plt.subplots(figsize=(10, 12))
sns.heatmap(cluster_means_norm[top_features].T, cmap='RdBu_r', center=0,
            xticklabels=[f'Cluster {i}' for i in cluster_means.index],
            cbar_kws={'label': 'Z-Score'})
ax.set_title('Top 30 Distinguishing Features')
plt.tight_layout()
plt.savefig(f'../../plots/{RUN}/cluster_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Detailed Analysis: Most Distinguishing Features

In [ ]:
top_5_features = cluster_means.var().nlargest(5).index.tolist()

fig, axes = plt.subplots(1, 5, figsize=(16, 4))
for idx, feat in enumerate(top_5_features):
    means = cluster_means[feat]
    axes[idx].bar(means.index, means.values, color=plt.cm.tab10(range(len(means))))
    axes[idx].set_xlabel('Cluster')
    axes[idx].set_ylabel('Mean')
    axes[idx].set_title(feat[:25] + '...' if len(feat) > 25 else feat, fontsize=9)

plt.suptitle('Top 5 Distinguishing Features', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.savefig(f'../../plots/{RUN}/top_features.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Demographic Analysis per Cluster

## 7. Mental Health Analysis per Cluster

## 8. Workplace Attitudes per Cluster

## 9. Cluster Profiles (Radar Chart)

## 10. Cluster Summary

## 11. Statistical Significance of Cluster Differences

## 12. Recommendations for HR